<a href="https://colab.research.google.com/github/shubhigupta991/Reddit-Flair-Detection/blob/main/scripts/Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
data = pd.read_csv("drive/MyDrive/data.csv")
data.head()

,title,id,score,url,num_of_comments,body,author,comments,flair,timestamp,combined_features
0,gelatine available,jk9zlt,0,https://www.reddit.com/r/india/comments/jk9zlt...,4,wish buy gelatine looking cheapest local price...,csstudentG,available regular kirana stores usually kept r...,AskIndia,2020-10-29 21:26:38,gelatine availableavailable regular kirana sto...
1,trevor noah jokes indo pak tensions seriousnes...,gv9lmh,37,https://www.reddit.com/r/india/comments/gv9lmh...,29,really watch much trevor noah late night show ...,CommYouNitty,mostly country really matter lives pretty chea...,AskIndia,2020-06-02 23:42:24,trevor noah jokes indo pak tensions seriousnes...
2,buying used bs4 scooty april 1 issues registra...,hpqq5o,11,https://www.reddit.com/r/india/comments/hpqq5o...,7,know right place ask askindia asking saw artic...,akza07,scammers keep scamming applies new vehicles bu...,AskIndia,2020-07-12 15:58:15,buying used bs4 scooty april 1 issues registra...
3,need feedback insurance policy took x post ask...,1s57oi,1,https://www.reddit.com/r/india/comments/1s57oi...,1,posting lack activity r askindia hello last ye...,dhavalcoholic,dear policy holder dhavalcoholic request help ...,AskIndia,2013-12-05 14:30:23,need feedback insurance policy took x post ask...
4,could suggest good dramedies,h7t0r2,5,https://www.reddit.com/r/india/comments/h7t0r2...,4,r askindia restricted posting sure ask hope gu...,dont_mess_with_tx,vicky donor jolly llb angoor comedy guide dram...,AskIndia,2020-06-13 04:58:52,could suggest good dramediesvicky donor jolly ...


In [5]:
data.fillna("None",inplace = True)

In [6]:
data.isna().sum()

title                0
id                   0
score                0
url                  0
num_of_comments      0
body                 0
author               0
comments             0
flair                0
timestamp            0
combined_features    0
dtype: int64

## Data Preprocessing and Model Fitting

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [8]:
def fit_and_score(models,X_train,y_train,X_test,y_test) :
    '''
    Fits and evaluate given machine learning model.
    
    Parameters:-
    models = Take a dictionary of models to fit and evaluate.
    X_train = Training data without labels.
    y_train = Training labels.
    X_test = Test data without labels.
    y_test = Test labels
    '''
    
    np.random.seed(21)
    model_scores = {}
    for model_name, model in models.items() :
        model = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', model)])
        print(f'Fitting {model_name} .....')
        model.fit(X_train,y_train)
        print(f'Evaluating {model_name} .....')
        model_scores[model_name] = model.score(X_test,y_test)
        
    return model_scores

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
models = {'LogisticRegression' : LogisticRegression(),
          'Linear_svm' : SGDClassifier(),
          'KNN': KNeighborsClassifier(),
          'RandomForestClassifier' : RandomForestClassifier(),
          'XGBClassifier' : XGBClassifier()}

In [ ]:
features = {'V' : data['combined_features'],
'W' : data['comments'],
'X' = data['title'],
'Y' = data['body'],
'Z' = data['url']}

 cat = data['flair']   

In [ ]:
for feature in features:
  scores = {}

  X_train,X_test,y_train,y_test = train_test_split(feature,cat,test_size=0.2, random_state = 21)
  print(f'Flair Detection using {feature} as Feature')
  model_scores = fit_and_score(models,X_train,y_train,X_test,y_test)

  scores[feature] =  model_scores